# Plotting and Analyzing Light Curves of Eclipsing Binary Stars with `lightkurve`

***

## Learning Goals

By the end of this tutorial, you will:

- Download and plot a light curve file using `astroquery.mast`.
- Download and plot a collection of the light curves using `lightkurve`.
- Create a periodogram of a collection of light curves to find the possible eclipsing binary period. 

This is Part 2 of a two-part tutorial. In a separate notebook for Part 1, `find_eclipse_candidates`, we used Vizier to select our binary star system candidate from a catalog created using the radial velocity method and confirmed that it had multiple TESS light curves in the MIT Quick-Look Pipeline ("QLP") archive. 

## Introduction

From Part 1 of this tutorial:

>The [Apache Point Observatory Galactic Evolution Experiment 2 (APOGEE-2)](https://www.sdss.org/surveys/apogee-2/)  observed near-infrared spectra of hundreds of thousands of stars in our Milky Way. This survey was used to search for stars with spectral lines showing the tell-tale radial velocity shifts associated with stellar binaries. Near the end of the APOGEE-2 survey, the [Transiting Exoplanet Survey Satellite (TESS)](https://tess.mit.edu/) was launched to search for exoplanets using the transit method; eclipses of primary stars by companion stars would also be observable by TESS. Finding a stellar system that has been observed by both surveys would provide confirmation of the properties of the system. 

>In this tutorial, we will load a catalog of binary stellar systems discovered through the radial velocity method and investigate if those systems also have light curves in the MAST archive from the TESS mission. Two sources for those already processed light curves are the TESS Data for Asteroseismology Lightcurves archive from TASOC (the TESS Asteroseismic Science Operations Center) or in the TESS Lightcurves From The MIT Quick-Look Pipeline ("QLP") archive. Though we'll focus on a narrow set of eclipsing binaries with previously-measured properties, the radial velocity catalog we'll be working with has nearly 5000 stellar systems that are yet to be fully characterized. 

Now, we'll actually view and analyze the light curves for just one of the radial velocity-selected stellar systems to see if we can observe an eclipse in the TESS data.

The workflow for this notebook consists of:
- Examining and Plotting the Light Curve FITS files
    - Exercise 1
- Combining Multiple Light Curve Products using lightkurve
    - Downloading the light curves
    - Plotting the light curves
- Finding the Period of the Variability using lightkurve
- Additional Exercises

## Imports

- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *astropy.coordinates SkyCoord* for creating sky coordinate objects
- *astropy.units* for units on physical quantities
- *matplotlib.pyplot* for plotting data
- *astroquery.mast Observations* for querying MAST for observations
- *lightkurve* for plotting and analying TESS light curves from the QLP or TASOC archives hosted on MASS

You may need to [install the ```lightkurve``` package using its quickstart guide](https://docs.lightkurve.org/quickstart.html). 

In [ ]:
%matplotlib inline
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
import matplotlib.pyplot as plt
from astroquery.mast import Observations
import lightkurve as lk

***

## Examining and Plotting the Light Curve .fits File

In Part 1 of the tutorial, we decided to investigate the binary star system with APOGEE ID 2M00085727+7341257 located at RA = 2.238636 degrees, Dec = 73.690475 degrees. The TIC (TESS Input Catalog) ID for this object is 373283207. The period determined by the analysis of [Price-Whelan et al. 2018 (AJ, 156, 1, 18)](https://ui.adsabs.harvard.edu/abs/2018AJ....156...18P/abstract) was 83.66193 days. For now, we'll enter the coordinate information manually and then use astroquery to search for the QLP observations

In [ ]:
coord=SkyCoord(ra=2.238636*u.degree, dec=73.690475*u.degree, frame='icrs')
per=83.66193
obs_tbl = Observations.query_region(coord,radius=0.008)
obs_tbl=obs_tbl[obs_tbl['provenance_name']=='QLP']
print(obs_tbl)

Our ```obs_tbl``` table from ```astroquery``` includes a column called "dataURL." With this URL, we can directly download the light curve file in .fits format to the harddrive. If it is successfully, 'COMPLETE' will appear as the first output. It will automatically download into the current working directory, unless the ```local_path``` option is set. Also, by default, the file will not download again if the file is already found on the harddrive. To see how to change this and other settings you can [view the ```Observations.download_file``` documentation](https://astroquery.readthedocs.io/en/latest/api/astroquery.mast.ObservationsClass.html#astroquery.mast.ObservationsClass.download_file). To download multiple files at once, use ```Observations.download_products``` instead.  

In [ ]:
url=obs_tbl[0]['dataURL']
Observations.download_file(url)

We can use ```fits``` from ```astropy``` to examine the light curve file. The filename will be the last part of the url string, when separated by slashes.

In [ ]:
filename=url.split('/')[-1]
lcfits=fits.open(filename)
lcfits.info()

The TESS light curve file has two extensions:

- No. 0 (Primary): This HDU (Header Data Unit) contains metadata related to the entire file.
- No. 1 (Light Curve): This HDU contains a binary table that holds data like flux measurements and times. We will extract information from here when we define the parameters for the light curve plot.

We can preview the contents of the second file extension by examining the names of the columns and the first few lines of data.

In [ ]:
print(lcfits[1].columns)
print(lcfits[1].data[0:5])

Detailed information about the QLP data products can be found linked at the [MAST QLP website](https://archive.stsci.edu/hlsp/qlp) in the [QLP Data Production Description Document](https://archive.stsci.edu/hlsps/qlp/hlsp_qlp_tess_ffi_all_tess_v1_data-prod-desc.pdf), a supplement to the RNAAS article ([Kunimoto et al., 2021, RNAAS, 5, 10, 234](https://ui.adsabs.harvard.edu/abs/2021RNAAS...5..234K/abstract)) that describes the process for creating the QLP light curves. 

At its most basic, a light curve is a plot of flux on the vertical axis vs. time on the horizontal axis. We can use ```matplotlib``` to take a very quick look at the data, but first you likely noted that there are two different columns of fluxes in the FITS data. As described in [Kunimoto et al. 2021](https://ui.adsabs.harvard.edu/abs/2021RNAAS...5..234K/abstract), SAP_FLUX is the un-detrended flux from the optical aperture and KSPSAP_FLUX is the detrended flux. As we can see below, from plotting the two types of fluxes, the detrended light curve (KSPSAP) filters out the long timescale variation; this long timescale variation is caused by intrinsic stellar variations, but we want to filter that out when searching for eclipses and transits. 

In [ ]:
plt.scatter(lcfits[1].data['TIME'],lcfits[1].data['SAP_FLUX'],label='SAP_FLUX',marker='.')
plt.scatter(lcfits[1].data['TIME'],lcfits[1].data['KSPSAP_FLUX'],label='KSPSAP_FLUX',marker='.')
plt.legend()

There are some notably low flux measurements, but they appear to be individual data points, not prolonged transits or eclipses. We will investigate these more below.

Is this enough information to find our eclipse? By eye, it looks like this light curve spans about 25 days. Let's confirm.

In [ ]:
print(np.max(lcfits[1].data['TIME'])-np.min(lcfits[1].data['TIME']))

The period of our stellar system as determined by the radial velocity method was about 84 days, but one individual QLP light curve only spans about 25 days. It would be best to make use of the multiple datasets available. As our analysis needs get more complicated, it's best to turn to a package that has our downloading, plotting, and analysis tools built in: `lightkurve`.

### Exercise 1

Use the documentation linked above and the [FITS Headers](https://docs.astropy.org/en/stable/io/fits/usage/headers.html) documentation from astropy to answer the following questions:
1. Is the magnitude of this target included, and if so, what is it?
2. Is the mass of this target included, and if so, what is it? 
3. What was the mass determined in the RV catalog? (You'll need to go back to Part 1 of the tutorial to complete this step!)

In [ ]:
# Place for code for Exercise 1

## Combining Multiple Light Curve Products using lightkurve
The Lightkurve package can also search the MAST archive and download all light curve products associated with a target stellar system. As we'll see, this package is built specifically to plot and work with light curves in ways that will be easier than the `matplotlib` plotting we were doing above. This portion of the tutorial is based on lightkurve's [Combining multiple quarters of Kepler data](https://docs.lightkurve.org/tutorials/2-creating-light-curves/2-1-combining-multiple-quarters.html) tutorial.

In [ ]:
search_result=lk.search_lightcurve(coord,author="QLP")
print(search_result)

### Downloading the light curves

In [ ]:
lc_collection=search_result.download_all()
lc_collection

By choosing download_all, we create `LightCurveCollection` (basically, an array) of LightCurve products. By examining the first index in the collection, we can see that the columns are the same as in the .fits file that we downloaded thorugh `astroquery`, above. 

In [ ]:
lc_0=lc_collection[0]
lc_0

### Plotting the light curves
We can use lightkurve's built in plotting feature to assure ourselves that we have downloaded the same dataset as we did above with ```astroquery```. This also helps us to confirm that by default, the lightkurve plotting function uses the ```sap_flux``` on the flux axis, rather than the detrended ```kspsap_flux```; the ```sap_flux``` can be used for studies of stellar variability, but these signals need to be removed from the light curve for detailed studies of eclipses and transits.

In [ ]:
lc_0.plot()

We must be sure to specify that we want to use ```kspsap_flux``` from now on, as below.

In [ ]:
lc_0.plot(column='kspsap_flux')

To combine all datasets as one, we can use the built-in stitching method.

In [ ]:
lc_stitched=lc_collection.stitch()
lc_stitched.plot(column='kspsap_flux')

Examining the full amount of data available, we can see that the target of interest was observed in two TESS Sectors back-to-back (Sectors 18 and 19) and then again in Sectors 24 and 25, with a gap of approximately 100 days between the end of Sector 19 and the beginning of Sector 24. These sector numbers were listed in ```search_result``` above. 

It's tempting to look at some of these dips as transits or eclipses in the data! However, the non-zero values in the `quality` column are worth investigating. The tutorial [Instrumental Noise in Kepler and K2 /#1: Data Gaps and Quality Flags](https://docs.lightkurve.org/tutorials/2-creating-light-curves/2-2-kepler-noise-1-data-gaps-and-quality-flags.html), though specific to Kepler and not TESS, is still useful to consult. It provides a way to investigate "decode" each flag. In our case, we want to reference the [QLP Data Production Description Documents](https://archive.stsci.edu/hlsps/qlp/hlsp_qlp_tess_ffi_all_tess_v1_data-prod-desc.pdf) to decode the flags after we review which unique ones are in our dataset and how prevalent they are.

In [ ]:
for flag in np.unique(lc_stitched.quality):
    print('Number of datapoints with flag '+str(flag)+': '+str(np.sum(lc_stitched.quality==flag)))

From the above, we can see that about 17% of our rows contain flag 4096, "Low precision points. The cadence was excluded because low precision data occurred in a large fraction of the stars" (from the documentation linked above). What do our light curves look like if we only plot the rows whose quality flag = 0?

In [ ]:
plt.scatter(lc_stitched['time'][lc_stitched['quality']==0].value,
         lc_stitched['kspsap_flux'][lc_stitched['quality']==0],marker='.')

As suspected, the low flux measurements are no longer visible, as they were spurious individual data points rather than transits or eclipses. We'll still continue to use the tools in `lightkurve` to discover if there are any periodic changes to the light curve that might not be noticeable to our eyes. 

## Finding the Period of the Variability using lightkurve

Could the same 84 day period computed with the APOGEE dataset be present in our data?

`lightkurve` has built-in methods for finding the periodicity embedded in the light curve; the result is what's known as a periodogram. This portion of the tutorial is based off of the [What are Periodogram objects?](https://docs.lightkurve.org/tutorials/1-getting-started/what-are-periodogram-objects.html#What-are-Periodogram-objects?) and [Creating periodograms and identifying significant peaks](http://docs.lightkurve.org/tutorials/3-science-examples/periodograms-creating-periodograms.html) tutorials; the second link in particular has more background information about the mathematics behind periodograms.

By default, the methods below will use the lc_stitched.flux property. As we saw above, by default, this is the ```sap_flux``` column and it includes some data points with non-zero quality flags. Before we continue, we will only choose datapoints whose quality flag is zero. We will then use [select_flux](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.select_flux.html) to use the ```kspsap_flux``` column instead (and likewise, the ```kspsap_flux_err``` column for error) when we use the light curve object from now on.

Next, we create the periodogram object (```pg```) and plot it.

In [ ]:
lc_stitched=lc_stitched[lc_stitched['quality']==0] 
lc_stitched.select_flux("kspsap_flux",flux_err_column='kspsap_flux_err').plot()

In [ ]:
pg=lc_stitched.select_flux("kspsap_flux",flux_err_column='kspsap_flux_err').to_periodogram()
pg.plot()

For the types of periods we are interested in (measured in multiple days), the frequency will be very small, so the view above may not be insightful. Below, we  plot the periodogram with the period on the horizontal axis (instead of frequency) and with both axes in a logarithmic scale. 

On top of the plot, we'll include a vertical line that corresponds with the measured period from Price-Whelan+ 2018 and a dashed vertical line that corresponds to the "Period at Max Power" determined by the periodogram. 

In [ ]:
ax=pg.plot(view='period',scale='log')
ax.axvline(per,label='PW+ 2018 period')
ax.axvline(pg.period_at_max_power.value,label='Period at Max Power',linestyle='--')
ax.legend()
print('PW+ 2018 period: '+str(per)+' days')
print('Period at Max Power: '+str(pg.period_at_max_power))

The two periods do not match, and in fact, there is no clear periodicity or repeating signal in the light curve of our target.

One way to check the veracity of a period is to plot the folded light curve. In a folded light curve, phase (in Julian days) appears on the horizontal axis instead of time. In other words, the observations are placed by where they appear in the periodic nature of the change. We can try to view the folded light curve assuming an 83.7 day period.

In [ ]:
lc_stitched.select_flux("kspsap_flux",flux_err_column='kspsap_flux_err').fold(period=per).scatter()

Unfortunately, no clear eclipse is visible on that timescale. Though this example didn't yield an observable eclipse, there are many (nearly 5000!) candidate binary systems in the Price-Whelan et al. 2018 catalog that may have eclispes observed by TESS! The TASOC and QLP archives contain already processed light curves ready for exploration and discovery. 

## Additional Exercises

1. In Part 1 of the tutorial, `find _eclipse_candidates`, we created a for loop to search for TASOC and QLP observations using astroquery.mast Observations. Load the same `tbl` variable from that tutorial (a filtered version of [Price-Whelan et al. 2018 (AJ, 156, 1, 18)](https://ui.adsabs.harvard.edu/abs/2018AJ....156...18P/abstract) Table 4a, and create a loop to search for TASOC and QLP observation using lk.search_lightcurve instead. Which is faster? 

2. Choose another stellar system from PW+2018, plot the stitched light curve, and create the periodogram. Compare the period at maximum power found to the period derived from the radial velocity method.

In [ ]:
# Place for code for Additional Exercise 1

In [ ]:
# Place for code for Additional Exercise 2

## Citations

If you use `astropy`, `astroquery`, `lightkurve`, or `VizieR` for published research, please cite the
authors. Follow these links for more information about citing these tools:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* If you use astroquery, please cite the paper [Ginsburg, Sipőcz, Brasseur et al 2019](https://ui.adsabs.harvard.edu/abs/2019AJ....157...98G/abstract).
* [Citing `lightkurve`](http://docs.lightkurve.org/about/citing.html)
* [Citing `VizieR`](https://cds.unistra.fr/vizier-org/licences_vizier.html?#copyrightvizier)


## About this Notebook

**Author:** Julia Kamenetzky  
**Last Updated:** Sep 2022  
**Next Review:** Mar 2023

For support, please contact the Archive HelpDesk at archive@stsci.edu.

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 